# 基准方法：基于序列比对的反应推荐
> zhenkun.shi@tib.cas.cn  
> 2024-11-08

Basline methods by using sequence aligmnet method.   

## 1. Import Packages

In [7]:
import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../')
import pandas as pd
from config import conf as cfg
from tqdm.notebook import tqdm
import tools.bioFunctionLib as bfl
from pandarallel import pandarallel 
pandarallel.initialize() 

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2. Load expdata - 10 folds

In [8]:
print('Loading trainning datasets feather path ...')
train_feather_files =[
    f'{cfg.DIR_DATASET}validation/fold{fold_index}/train.feather'
    for fold_index in range(1,11)
]

print('Loading validation datasets feather path ...')
vali_feather_files = [
    f'{cfg.DIR_DATASET}validation/fold{fold_index}/valid.feather' 
    for fold_index in range(1, 11)
]

vali_res_blast = [
    f'{cfg.DIR_RES_BASELINE}results/direct_methods/blast/fold{item}.tsv' 
    for item in range(1, 11)
]

print('Loading EC2RXN map...')

Loading trainning datasets feather path ...
Loading validation datasets feather path ...
Loading EC2RXN map...


## 3. Blast - Direct

In [9]:
def blast_via_train_test(ds_train, ds_test):
    blast_res_rxn = bfl.getblast(train=ds_train[['uniprot_id', 'seq']], test=ds_test[['uniprot_id', 'seq']], k=30)
    blast_res_rxn= blast_res_rxn[['id','sseqid']].merge(ds_train[['uniprot_id', 'reaction_id']], left_on='sseqid', right_on='uniprot_id', how='left')[['id','reaction_id']].rename(columns={'id':'uniprot_id', 'reaction_id':'rxn_blast'})
    blast_res_rxn = blast_res_rxn.groupby('uniprot_id', as_index=False).agg({'rxn_blast': lambda x: ';'.join(set(x))})
    return blast_res_rxn

In [10]:
for i in tqdm(range(10)):
    ds_train = pd.read_feather(train_feather_files[i])
    ds_test = pd.read_feather(vali_feather_files[i])
    blast_res_rxn =  blast_via_train_test(ds_train, ds_test)
    blast_res_rxn = ds_test[['uniprot_id']].merge(blast_res_rxn, on='uniprot_id', how='left').fillna('NO-PREDICTION')
    blast_res_rxn.to_csv(f'{vali_res_blast[i]}', sep='\t', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]